In [192]:
from pathlib import Path
import configparser
from psycopg2 import connect
import pandas.io.sql as pandasql
from IPython.display import HTML
from psycopg2.extras import execute_values
import requests
from psycopg2.extras import RealDictCursor
import json
CONFIG = configparser.ConfigParser()
CONFIG.read(str(Path.home().joinpath('db.cfg')))
dbset = CONFIG['DBSETTINGS']
con = connect(**dbset)

In [ ]:
# Pull data from postgres

In [193]:
get_centreline_sql = '''SELECT 'Feature' as type,  json_build_object('gid', gid) as properties, ST_AsGeoJson(geom)::json as geometry FROM gis.centreline WHERE FCODE_DESC IN ('Collector','Collector Ramp','Expressway','Expressway Ramp',
'Local','Major Arterial','Major Arterial Ramp','Minor Arterial',
'Minor Arterial Ramp','Pending') ORDER BY gid LIMIT 100'''

with con:
    with con.cursor(cursor_factory=RealDictCursor) as cur:
        cur.execute(get_centreline_sql)
        results = cur.fetchall()
        final_json = json.dumps({"type": "FeatureCollection",
                                "features": results})


In [ ]:
# Send data to SharedStreets API

In [200]:
base_url = "https://api.sharedstreets.io/v0.1.0/match/geoms"
apiKey = "bdd23fa1-7ac5-4158-b354-22ec946bb575"
params = {'ignoreDirection': 'false',
          'bearingTolerance' : 45 ,
          'searchRadius' : 60,
          'auth' : apiKey,
          'tileHierarchy': 6,
          'dataSource': 'osm/planet-181029',
          'lengthTolerance' : 0.60,
          'snapTopology': 'true'
         }
headers= {"Content-type": "application/json; charset=UTF-8"}


In [201]:
r = requests.post(base_url, params = params, headers = headers, json = {"type": "FeatureCollection", "features": results})

In [202]:
returned_json = r.json()

In [208]:
feature = returned_json.get('matched').get('features')[0]

In [217]:
print('numrange(' + str(feature['properties']['section'][0]) +', '+ str(feature['properties']['section'][1]) +')')

numrange(0.39074573712912297, 14.153055239319883)


In [204]:
# Send row to database

In [268]:
features = returned_json.get('matched').get('features')
rows = []
for feature in features:
    row = (feature['properties']['referenceId'], 
           feature['properties']['fromIntersectionId'],
           feature['properties']['toIntersectionId'],
           feature['properties']['roadClass'], 
           feature['properties']['direction'],
           feature['properties']['referenceLength'],
           feature['properties']['side'], 
           feature['properties'].get('score', 'NULL'),
#            'numrange(' + str(feature['properties']['section'][0]) +', '+ str(feature['properties']['section'][1]) +')', 
           '[' + str(feature['properties']['section'][0]) +', '+ str(feature['properties']['section'][1]) +')', 
           str('ST_GeomfromGeojson(')+json.dumps(feature0['geometry'])+')', 
           feature['properties']['originalFeature']['properties']['gid'])
    rows.append(row)
    

In [269]:
sql='INSERT INTO natalie.test_table (reference_id, from_intersection, to_intersection, road_class, direction, reference_length, side, score,  section, geometry, gid) VALUES %s'

with con:
    with con.cursor() as cur:
        execute_values(cur, 
                       sql, 
                       rows)


InternalError: parse error - invalid geometry
LINE 1: ...nown','[0.39074573712912297, 14.153055239319883)','ST_Geomfr...
                                                             ^
HINT:  "ST" <-- parse error at position 2 within geometry


In [222]:
json.dumps(feature0['geometry'])

'{"coordinates": [[-79.54301264504791, 43.59272670361093], [-79.54305330000001, 43.592615800000004], [-79.54306049139689, 43.59260843398357]], "type": "LineString"}'

In [241]:
'numrange(' + str(feature0['properties']['section'][0]) +', '+ str(feature0['properties']['section'][1]) +')'

'numrange(0.39074573712912297, 14.153055239319883)'

In [207]:
for fruit in ['apple','banana', 'orange']:
    print(fruit)

apple
banana
orange


In [267]:
feature1['properties'].get('score', 'NULL')

0.6647517397903042

In [265]:
feature1 = features[1]
feature1['properties']['score']

0.6647517397903042